In [6]:
import requests
import pandas as pd
import time
import urllib3
from bs4 import BeautifulSoup

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

BASE_URL = "https://www.howstat.com/Cricket/statistics/players/PlayerProgressBat_ODI.asp"
PLAYER_ID = 3600

def get_html(url, params=None):
    return requests.get(url, params=params, verify=False, timeout=15).text

def extract_date_links(html):
    soup = BeautifulSoup(html, "html.parser")

    tables = soup.find_all("table")
    if not tables:
        print("❌ No tables found")
        return []

    matches = []

    for table in tables:
        for row in table.find_all("tr"):
            cells = row.find_all("td")
            if len(cells) < 2:
                continue

            # Date is usually the FIRST anchor in the row
            date_link = row.find("a", href=True)
            if not date_link:
                continue

            date = date_link.text.strip()
            href = date_link["href"]

            if "MatchScorecard" in href:
                match_url = "https://www.howstat.com/Cricket" + href
                matches.append((date, match_url))

    print(f"✅ Found {len(matches)} date links")
    return matches

def extract_result(match_url):
    html = get_html(match_url)
    soup = BeautifulSoup(html, "html.parser")

    for line in soup.get_text("\n").split("\n"):
        if "Result:" in line:
            return line.replace("Result:", "").strip()

    return "Result not found"

def main():
    html = get_html(BASE_URL, {"PlayerID": PLAYER_ID})

    date_links = extract_date_links(html)
    if not date_links:
        print("❌ No matches extracted")
        return

    data = []

    for i, (date, url) in enumerate(date_links, 1):
        print(f"[{i}/{len(date_links)}] {date}")
        result = extract_result(url)
        data.append({"Date": date, "Result": result})
        time.sleep(0.7)

    df = pd.DataFrame(data)
    df.to_csv("howstat_results.csv", index=False)
    print("✅ CSV saved: howstat_results.csv")

if __name__ == "__main__":
    main()


✅ Found 933 date links
[1/933] 18/08/2008
[2/933] 18/08/2008
[3/933] 18/08/2008
[4/933] 18/08/2008
[5/933] 18/08/2008
[6/933] 20/08/2008
[7/933] 24/08/2008
[8/933] 27/08/2008
[9/933] 29/08/2008
[10/933] 14/09/2009
[11/933] 26/09/2009
[12/933] 28/09/2009
[13/933] 30/09/2009
[14/933] 25/10/2009
[15/933] 02/11/2009
[16/933] 15/12/2009
[17/933] 18/12/2009
[18/933] 24/12/2009
[19/933] 27/12/2009
[20/933] 05/01/2010
[21/933] 07/01/2010
[22/933] 10/01/2010
[23/933] 11/01/2010
[24/933] 13/01/2010
[25/933] 21/02/2010
[26/933] 24/02/2010
[27/933] 27/02/2010
[28/933] 28/05/2010
[29/933] 30/05/2010
[30/933] 03/06/2010
[31/933] 05/06/2010
[32/933] 16/06/2010
[33/933] 19/06/2010
[34/933] 22/06/2010
[35/933] 24/06/2010
[36/933] 16/08/2010
[37/933] 25/08/2010
[38/933] 28/08/2010
[39/933] 20/10/2010
[40/933] 28/11/2010
[41/933] 01/12/2010
[42/933] 04/12/2010
[43/933] 07/12/2010
[44/933] 10/12/2010
[45/933] 12/01/2011
[46/933] 15/01/2011
[47/933] 18/01/2011
[48/933] 21/01/2011
[49/933] 23/01/2011
[50/93

KeyboardInterrupt: 

In [5]:
df.head()

NameError: name 'df' is not defined

In [2]:
# !pip install requests beautifulsoup4